In [1]:
import sys
import os

# プロジェクトルートを Python パスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import numpy as np
from core.linalg_utils import random_unitary_matrix
from graph.prior.gaussian_prior import GaussianPrior
from graph.propagator.unitary_propagator import UnitaryPropagator
from graph.measurement.gaussian_measurement import GaussianMeasurement

In [9]:
# 1. グラフ構築（X = GaussianPrior(); Y = U @ X; Measurement）
n = 8
U = random_unitary_matrix(n, seed=42)

# 真のデータと観測値を準備
true_x = np.ones((n,), dtype=np.complex128)
true_y = U @ true_x
noise = np.random.normal(scale=0.1, size=(n,)) + 1j * np.random.normal(scale=0.1, size=(n,))
observed = true_y + noise

# グラフ構築
X = GaussianPrior(shape=(n,), mean=0.0, var=1.0, seed=123)
Y = UnitaryPropagator(U) @ X
measurement = GaussianMeasurement(Y, observed_array=observed, var=0.01)

In [14]:
# 1. Forward pass
X.parent.forward()     # Prior → X
X.forward()            # X → UnitaryPropagator
Y.parent.forward()
Y.forward()            # Ux → Y

# 2. Backward pass
measurement.backward() # Measurement → Y
Y.backward()           # Y → UnitaryPropagator
Y.parent.backward()
X.backward()           # U†y → X

# 3. Compute belief at X
x_belief = X.compute_belief()

# 4. 結果表示
print("X belief mean:\n", x_belief.data)
print("X belief precision:\n", x_belief.precision)

X belief mean:
 [0.9726733 +0.02267031j 0.96381884+0.02602988j 0.92152467+0.01257714j
 1.20903688-0.08806878j 1.07980163-0.11212723j 1.11169636+0.04237791j
 0.96141277-0.09707071j 1.10886572-0.05603513j]
X belief precision:
 [101. 101. 101. 101. 101. 101. 101. 101.]
